In [1]:
import os
import bs4
from bs4 import BeautifulSoup as soup
import pandas as pd
import spacy
import re
import en_core_web_sm


In [2]:
def extract_html(input_html):
    with open(input_html) as inf:
        txt = inf.read()
    soup_out = soup(txt, "html.parser")
    print(input_html)
    return soup_out

In [3]:
def extract_firm(input_soup):
    temp = input_soup.find(id='article_participants')
    for element in temp.findAll('a'):
        if element.get('title') != None :
            return(str(element.get('title')))   
    return None

In [4]:
def extract_ticker(input_soup):
    temp = input_soup.find(id='article_participants')
    for element in temp.findAll('a'):
        if element.get('title') != None :
            return(str(element.contents[0].strip()))   
    return None

In [5]:
def extract_time(input_soup):
    temp = input_soup.find(id='article_participants')
    try:
        for element in temp.findAll('p'):
            if (pd.notnull(element.text) and element.text!=''):
                if( re.search('call',element.contents[0],flags=re.IGNORECASE)):
                    call_title = element.contents[0].split()
                    return str(call_title[0]),str(call_title[1])
        return None,None
    except:
        return None,None
    
def extract_quarter(input_soup):
    temp = input_soup.find(id='article_participants')
    try:
        for element in temp.findAll('p'):
            if (pd.notnull(element.text) and element.text!=''):
                if( re.search('call',element.contents[0],flags=re.IGNORECASE)):
                    call_title = element.contents[0].split()
                    return str(call_title[0])
        return None
    except:
        return None
    
def extract_year(input_soup):
    temp = input_soup.find(id='article_participants')
    try:
        for element in temp.findAll('p'):
            if (pd.notnull(element.text) and element.text!=''):
                if( re.search('call',element.contents[0],flags=re.IGNORECASE)):
                    call_title = element.contents[0].split()
                    return str(call_title[1])
        return None
    except:
        return None

In [6]:
def test_analyst(input_soup):
    temp = input_soup.find(id='article_participants')
    for element in temp.findAll('strong'):
        if(re.search('analyst',element.contents[0],flags=re.IGNORECASE)):
            return(True)
    return(None)


In [7]:
def extract_analysts(input_soup):
    temp = input_soup.find(id='article_participants')
    test_string = []
    print_flag = False
    for element in temp.find_all('p'):
        if(re.search('analyst',element.text,flags=re.IGNORECASE)):
            #print(element.text.strip(),'###')
            element = element.find_next()
            while  (1):
                element = element.find_next()
            #print(element.contents[0].strip())
            #print(element.text.strip())
                if element.find('strong') or \
                element.find('div'):
            #print('FOUND')
                    break
                if (pd.notnull(element.text) and element.text!=''):
                    test_string.append(element.contents[0].strip())    
            break
    return test_string

In [8]:
def extract_executives(input_soup):
    test_string = []
    temp = input_soup.find(id='article_participants')
    # print(soup_out)
    print_flag = False
    for element in temp.find_all('p'):
        if(re.search('executive',element.text,flags=re.IGNORECASE)):
            #print(element.text.strip(),'###')
            element = element.find_next()
            while  (1):
                element = element.find_next()
                # print(element.contents[0])
                # print(element.text.strip())
                if element.find('strong') or \
                element.find('div'):            
                #print('FOUND')
                    break
                test_string.append(element.text.strip())
        
            break
    return test_string

In [9]:
def is_analyst_present(analyst_list):
    if analyst_list: 
        return True
    else: 
        return False

In [10]:
def extract_name (input_string,delim='-'):
    if not pd.isnull(input_string):
        name_search = re.search('(^[^-]*)-', input_string, re.IGNORECASE)
        try:
            return name_search.group(1).strip()
        except:
            return None        
    else:
        return None
def extract_title (input_string,delim='-'):
    if not pd.isnull(input_string):
        title_search = re.search('^[^-]*-(.*)', input_string, re.IGNORECASE)
        try:
            return title_search.group(1).strip()
        except:
            return None           
        else:
            return None
    else:
        return None

In [11]:
def extract_content(input_soup,current_name):
#     current_name = 'Panna Sharma'
    if not (pd.isnull(current_name) ):
        temp = input_soup.find(id='article_content')
        test_string = []
        # print(temp)
        for element in temp.find_all('strong'):
            if (re.search(current_name.strip().lower(),element.text,flags=re.IGNORECASE)):
                while  (1):
                    element = element.find_next()
        #           print(element.text.strip())
                    if element.find('strong') or \
                    element.find('div'):
                        break
                    test_string.append(element.text.strip())    
        return test_string
    else:
        return None

In [12]:
def extract_qanda(input_soup,current_name):
#     current_name = 'Panna Sharma'
    if not (pd.isnull(current_name) ):
        temp = input_soup.find(id='article_qanda')
        test_string = []
        # print(temp)
        if temp.find_all('span'):
            for element in temp.find_all('span'):
                if (re.search(current_name.strip().lower(),element.text,flags=re.IGNORECASE)):
                    while  (1):
                        element = element.find_next()
            #           print(element.text.strip())
                        if element.find('strong') or \
                        element.find('div') or \
                        element.find('span'):
                            break
                        test_string.append(element.text.strip())    
        else:                
            for element in temp.find_all('strong'):
                if (re.search(current_name.strip().lower(),element.text,flags=re.IGNORECASE)):
                    while  (1):
                        element = element.find_next()
            #           print(element.text.strip())
                        if element.find('strong') or \
                        element.find('div') or \
                        element.find('span'):
                            break
                        test_string.append(element.text.strip())      
            
        return test_string
    else:
        return None

In [13]:
nlp = en_core_web_sm.load()

def basic_wc(input_text):
    if not (input_text==[]):
#         decoded_txt=input_text.decode('utf-8')
        sent =  " ".join(str(x) for x in input_text)
        sent = re.sub(pattern='[^a-zA-Z0-9 -]',repl='',string=sent)            
        doc = nlp(sent)
        return doc.__len__()
    else:
        return 0


In [14]:
def extract_chunks(input_text):
    sent =  " ".join(str(x) for x in input_text)
    #     doc = nlp(input_text)
    doc = nlp(sent)
    spans = list(doc.ents)  #+ list(doc.noun_chunks)
    out_list = []
    for span in spans:
        span.merge()
        out_list.append(span)
    return out_list

In [15]:
def extract_name_pair(name, input_text):
    out_list = []
    if not (input_text==[]):
#         s_out = extract_chunks(input_text)  
        sent =  " ".join(str(x) for x in input_text)
        #     doc = nlp(input_text)
        doc = nlp(sent)        
#         doc = nlp(input_text)
        spans = list(doc.ents)  #+ list(doc.noun_chunks)

        for item in spans:
            try:
                item = re.sub(pattern='[^a-zA-Z0-9 -]',repl='',string=item.string)            
                if (re.search(item,name,flags=re.IGNORECASE)):
                    out_list.append(item)

            except:
                pass
            
    return out_list
#     else:
#         return None

In [16]:
collection = "html"

# n_test=50
file_list = os.listdir(collection)
# for i in range(len(file_list)):
# for i in range(n_test):
#     print(file_list[i])
print(len(file_list))

91979


In [17]:
collection = "html"

n_test=4
file_list = os.listdir(collection)
out_list =[]
# for i in range(len(file_list)):
print('FILE\tANALYST_Flag\tFIRM\tQuarter\tYear') 

for i in range(n_test):
    with open(collection + '/' + file_list[i]) as inf:
        if(re.search('html',file_list[i],flags=re.IGNORECASE)):
            out_list.append(file_list[i])
        

FILE	ANALYST_Flag	FIRM	Quarter	Year


In [27]:
out_pd = pd.DataFrame(out_list)
out_pd.columns=['file']
out_pd['text'] = out_pd.apply(lambda x: extract_html('html/'+x['file']), axis=1)
out_pd['quarter'] = out_pd.apply(lambda x: extract_quarter(x['text']), axis=1)
out_pd['year'] = out_pd.apply(lambda x: extract_year(x['text']), axis=1)
# out_pd['quarter_year'] = out_pd.apply(lambda x: extract_q(x['text']), axis=1)
out_pd['executive_firm'] = out_pd.apply(lambda x: extract_firm(x['text']), axis=1)
out_pd['analyst_present'] = out_pd.apply(lambda x: test_analyst(x['text']), axis=1)
out_pd['ticker'] = out_pd.apply(lambda x: extract_ticker(x['text']), axis=1)

out_pd['analysts'] = out_pd.apply(lambda x: extract_analysts(x['text']), axis=1)
out_pd['execs'] = out_pd.apply(lambda x: extract_executives(x['text']), axis=1)

s = out_pd.execs.apply(pd.Series)\
              .stack()\
              .reset_index(level=1, drop=True)\
              .to_frame('executive')
out_executives = out_pd.join(s)                      


out_executives['exec_name'] = out_executives.apply(lambda x: extract_name(x['executive']), axis=1)
out_executives['exec_title'] = out_executives.apply(lambda x: extract_title(x['executive']), axis=1)

out_executives['executive_content'] = out_executives.apply(lambda x: \
                                         extract_content(x['text'],x['exec_name']), axis=1)
out_executives['executive_qanda'] = out_executives.apply(lambda x: \
                                         extract_qanda(x['text'],x['exec_name']), axis=1)

out_executives['exec_qna_count'] = out_executives.apply(lambda x: \
                                         basic_wc(x['executive_qanda']), axis=1)

out_executives['exec_content_count'] = out_executives.apply(lambda x: \
                                         basic_wc(x['executive_content']), axis=1)
out_executives['exec_total_count'] = out_executives.exec_qna_count + out_executives.exec_content_count

out_executives=out_executives.reset_index(level=0, drop=True)
# out_analyst
s = out_executives.execs.apply(pd.Series)\
              .stack()\
              .reset_index(level=1, drop=True)\
              .to_frame('mention_name')
out_executives = out_executives.join(s)          
out_executives['mention_name'] = out_executives.apply(lambda x: extract_name(x['mention_name']), axis=1)

out_executives['exec_mention_qna'] = out_executives.apply(lambda x: extract_name_pair(x['mention_name'],x['executive_qanda']), axis=1)
out_executives['exec_mention_transcript'] = out_executives.apply(lambda x: extract_name_pair(x['mention_name'],x['executive_content']), axis=1)
out_executives['exec_count_qna'] = out_executives.apply(lambda x: \
                              len(x['exec_mention_qna']), axis=1)
out_executives['exec_count_transcript'] = out_executives.apply(lambda x: \
                              len(x['exec_mention_transcript']), axis=1)
out_executives['exec_count_total']= out_executives.exec_count_qna+out_executives.exec_count_transcript


html/1.html
html/10.html
html/100.html


In [28]:
# out_analyst.drop(['text','analyst_present','analysts','executive'],axis=1)
out_executives.drop(['text','executive','execs'],axis=1)\
.to_csv('analyst_out_foringestion.csv',sep=',', encoding = 'utf-8')
# out_executives.select({[}'file','quarter_yer'])

In [ ]:
import csv
fieldnames = ['index','fname', 'status']

with open('log_extract.csv', 'w') as csvfile:
    logwriter = csv.writer(csvfile,delimiter='\t',quotechar='"',quoting=csv.QUOTE_MINIMAL)
#     logwriter.writerow(fieldnames)

 
    for ii in range(1,91980):
#     for ii in range(1,4):
        try:
            total_rows = len(file_list)
            row_start = ii*1
            row_end = (ii*1)+1
        #     print('from: ', row_start, ' to: ', row_end)
            filename = 'exec/exec_out_foringestion_'+str(row_start)+'.csv'
#             print('FILE\tANALYST_Flag\tFIRM\tQuarter\tYear') 
#             print(ii,'####')
            out_list =[]

            for i in range(row_start,row_end):
                with open(collection + '/' + file_list[i]) as inf:
                    if(re.search('html',file_list[i],flags=re.IGNORECASE)):
                        out_list.append(file_list[i])

            out_pd = pd.DataFrame(out_list)
            out_pd.columns=['file']
            out_pd['text'] = out_pd.apply(lambda x: extract_html('html/'+x['file']), axis=1)
            out_pd['quarter'] = out_pd.apply(lambda x: extract_quarter(x['text']), axis=1)
            out_pd['year'] = out_pd.apply(lambda x: extract_year(x['text']), axis=1)
            # out_pd['quarter_year'] = out_pd.apply(lambda x: extract_q(x['text']), axis=1)
            out_pd['executive_firm'] = out_pd.apply(lambda x: extract_firm(x['text']), axis=1)
            out_pd['ticker'] = out_pd.apply(lambda x: extract_ticker(x['text']), axis=1)

            out_pd['analysts'] = out_pd.apply(lambda x: extract_analysts(x['text']), axis=1)
            out_pd['execs'] = out_pd.apply(lambda x: extract_executives(x['text']), axis=1)

            s = out_pd.execs.apply(pd.Series)\
                          .stack()\
                          .reset_index(level=1, drop=True)\
                          .to_frame('executive')
            out_executives = out_pd.join(s)                      


            out_executives['exec_name'] = out_executives.apply(lambda x: extract_name(x['executive']), axis=1)
            out_executives['exec_title'] = out_executives.apply(lambda x: extract_title(x['executive']), axis=1)

            out_executives['executive_content'] = out_executives.apply(lambda x: \
                                                     extract_content(x['text'],x['exec_name']), axis=1)
            out_executives['executive_qanda'] = out_executives.apply(lambda x: \
                                                     extract_qanda(x['text'],x['exec_name']), axis=1)

            out_executives['exec_qna_count'] = out_executives.apply(lambda x: \
                                                     basic_wc(x['executive_qanda']), axis=1)

            out_executives['exec_content_count'] = out_executives.apply(lambda x: \
                                                     basic_wc(x['executive_content']), axis=1)
            out_executives['exec_total_count'] = out_executives.exec_qna_count + out_executives.exec_content_count

            out_executives=out_executives.reset_index(level=0, drop=True)
            # out_analyst
            s = out_executives.execs.apply(pd.Series)\
                          .stack()\
                          .reset_index(level=1, drop=True)\
                          .to_frame('mention_name')
            out_executives = out_executives.join(s)          
            out_executives['mention_titile'] = out_executives.apply(lambda x: extract_title(x['mention_name']), axis=1)
            out_executives['mention_name'] = out_executives.apply(lambda x: extract_name(x['mention_name']), axis=1)
            out_executives['exec_mention_qna'] = out_executives.apply(lambda x: extract_name_pair(x['mention_name'],x['executive_qanda']), axis=1)
            out_executives['exec_mention_transcript'] = out_executives.apply(lambda x: extract_name_pair(x['mention_name'],x['executive_content']), axis=1)
            out_executives['mention_name_count_qna'] = out_executives.apply(lambda x: \
                                          len(x['exec_mention_qna']), axis=1)
            out_executives['mention_name_count_transcript'] = out_executives.apply(lambda x: \
                                          len(x['exec_mention_transcript']), axis=1)
            out_executives['mention_name_count_total']= out_executives.mention_name_count_qna+out_executives.mention_name_count_transcript
#             print('here##')
            # out_analyst.drop(['text','analyst_present','analysts','executive'],axis=1)
            out_executives.drop(['text','executive','execs','exec_mention_qna',\
                                 'exec_mention_transcript'],axis=1)\
            .to_csv(filename,sep=',', encoding = 'utf-8',header=False)
            # out_executives.select({[}'file','quarter_yer'])    
            logwriter.writerow([ii,out_executives.file.head(1).item(),'success'])
        except:
            print('exception: ', ii, ' ###')
            logwriter.writerow([ii,out_executives.file.head(1).item(),'fail'])
            
            pass

html/1.html
html/10.html
html/100.html
html/1000.html
html/10000.html
html/10001.html
html/10002.html
exception:  7  ###
html/10003.html
html/10004.html
html/10005.html
html/10006.html
html/10007.html
html/10008.html
html/10009.html
html/1001.html
html/10010.html
exception:  16  ###
html/10011.html
html/10012.html
html/10013.html
exception:  19  ###
html/10014.html
html/10015.html
exception:  21  ###
html/10016.html
html/10017.html
html/10018.html
html/10019.html
html/1002.html


In [ ]:
import csv
fieldnames = ['index','fname', 'status']

with open('log_extract.csv', 'wb') as csvfile:
    writer = csv.DictWriter(csvfile,delimiter='\t',quotechar='"',quoting=csv.QUOTE_NONE, fieldnames=fieldnames)
    writer.writer(['test','test','success'])

In [40]:
out_analyst.file.head(1).item()


'1.html'